In [148]:
import gzip
from collections import defaultdict
import re
import numpy
import matplotlib.pyplot as plt
import sklearn.linear_model as skl



def readGz(f):
  for l in gzip.open(f):
    yield eval(l)

reviews = []

for l in readGz("train.json.gz"):
    reviews.append(l)

reviews_train = reviews[:100000]
reviews_validation = reviews[100000:]

In [182]:
def trainAlpha(userToItemDict,betaU,betaI):
    alphaSum = 0.0
    for userId in userToItemDict:
        for itemId in userToItemDict[userId]:
            alphaSum += userToItemDict[userId][itemId] - (betaU[userId] + betaI[itemId])
    # alpha = float(alphaSum) / float(len(reviews_train))
    alpha = float(alphaSum) / float(len(reviews))
    return alpha

def trainBetaU(userToItemDict, alpha, betaU, betaI):
    for userId in userToItemDict:
        betaUSum = 0.0
        for itemId in userToItemDict[userId]:
            betaUSum += userToItemDict[userId][itemId] - (alpha + betaI[itemId])
        betaU[userId] = float(betaUSum) / float(lam + len(userToItemDict[userId]))
    return betaU


def trainBetaI(itemToUserDict,alpha, betaU, betaI):
    for itemId in itemToUserDict:
        betaISum = 0.0
        for userId in itemToUserDict[itemId]:
            betaISum += itemToUserDict[itemId][userId] - (alpha + betaU[userId])
        betaI[itemId] = float(betaISum) / float(lam + len(itemToUserDict[itemId]))
    return betaI

def trainGammaU(userToItemDict, alpha, betaU, betaI):
    for userId in userToItemDict:
        betaUSum = 0.0
        for itemId in userToItemDict[userId]:
            betaUSum += userToItemDict[userId][itemId] - (alpha + betaI[itemId])
        betaU[userId] = float(betaUSum) / float(lam + len(userToItemDict[userId]))
    return betaU

def getMSESum(userToItemDict,alpha,betaU,betaI):
    errorSum = 0.0
    for userId in userToItemDict:
        for itemId in userToItemDict[userId]:
            if userId not in betaU:
                betaU[userId] = 0.0

            if itemId not in betaI:
                betaI[itemId] = 0.0

            prediction = alpha + betaU[userId] + betaI[itemId]
            error = prediction - userToItemDict[userId][itemId]
            errorSum += error ** 2
    return errorSum

def trainModel(userToItemDict, itemToUserDict, alpha, betaU, betaI, lam):
    objectiveSum  = 100000
    newObjectiveSum = 1
    steps = 100
    while steps > 0:
        steps -= 1
        # Train alpha
        alpha = trainAlpha(userToItemDict,betaU,betaI)

        # Train Beta U
        betaU = trainBetaU(userToItemDict,alpha, betaU, betaI)

        # Train Beta I
        betaI = trainBetaI(itemToUserDict,alpha, betaU, betaI)

        Check convergence
        newErrorSum = getMSESum(userToItemDict,alpha,betaU,betaI)
        errorSum = getMSESum(userToItemDict,alpha,betaU,betaI)

        betaUSum = 0.0
        for userId in userToItemDict:
            betaUSum += betaU[userId] ** 2

        betaISum = 0.0
        for itemId in itemToUserDict:
            betaISum += betaI[itemId] ** 2

        newObjectiveSum = errorSum + lam*(betaUSum + betaISum)
        diff = objectiveSum - newObjectiveSum
        if diff < 0.0001:
            break
        objectiveSum = newObjectiveSum
        print(alpha)
    return alpha, betaU, betaI, newObjectiveSum

In [172]:
# Training rating(u,i) = alpha + betaU + betaI

userToItemDict = defaultdict(dict)
itemToUserDict = defaultdict(dict)
# for review in reviews_train:
for review in reviews:
    userId, itemId = review['reviewerID'],review['itemID']
    userToItemDict[userId][itemId] = review['rating']
    itemToUserDict[itemId][userId] = review['rating']

In [184]:
totalRating = sum([x['rating'] for x in reviews])
averageRating = totalRating/len(reviews)

In [185]:
# initializing random alpha and beta values
alpha = averageRating
betaU = {userId:0.0 for userId in userToItemDict}
betaI = {itemId:0.0 for itemId in itemToUserDict}
# lam = 1
lam = 6
alpha, betaU, betaI ,objectiveSum = trainModel(userToItemDict, itemToUserDict, alpha,betaU,betaI, lam)

In [161]:
userToItemDict_validation = defaultdict(dict)
itemToUserDict_validation = defaultdict(dict)
for review in reviews_validation:
    userId, itemId = review['reviewerID'],review['itemID']
    userToItemDict_validation[userId][itemId] = review['rating']
    itemToUserDict_validation[itemId][userId] = review['rating']
    
errorSum = getMSESum(userToItemDict_validation,alpha,betaU,betaI)
mse = errorSum / len(reviews_validation)

QN 6
Mean Squared Error =  1.1396117754860262


In [146]:
# TRAINING lambda
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
lam = 1
alpha = 0
betaU = {userId:0.0 for userId in userToItemDict}
betaI = {itemId:0.0 for itemId in itemToUserDict}
objectiveSum = 100000
while True:
    alpha, betaU, betaI ,newObjectiveSum = trainModel(userToItemDict_validation, itemToUserDict_validation, alpha,betaU,betaI, lam)
    print(newObjectiveSum)
    if newObjectiveSum > objectiveSum:
        break
    newObjectiveSum = objectiveSum
    lam += 0.5

KeyError: 'I238897042'

In [188]:
# Rating Test

avgBetaU = sum(betaU.values()) / len(betaU)
avgBetaI = sum(betaI.values()) / len(betaI)

index = 0
predictions = open("predictions_Rating.txt", 'w')
for l in open("pairs_Rating.txt"):
    if l.startswith("userID"):
        #header
        predictions.write(l)
        continue
    u,i = l.strip().split('-')
    if u not in betaU:
        # betaU[u] = 0.0
        betaU[u] = avgBetaU
    if i not in betaI:
        # betaI[i] = 0.0
        betaI[i] = avgBetaI
    prediction = alpha + betaU[u] + betaI[i]
    predictions.write(u + '-' + i + ',' + str(prediction) + '\n')
    index += 1
predictions.close()